<a href="https://colab.research.google.com/github/aspav90/Math-5670-assignment-3/blob/master/group_1_assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Assignment 3

Oleksandr Pavlenko \\
 \\
in what follows the ***requests*** library is used to receive XML data from the website as a string. The ***xlsxwriter*** library is used to write the resulting array of rates to an Excel file. The ***lxml.etree*** module is used to parse the XML data.

I know that this assigment can be done more efficiently using ***pandas*** or ***numpy*** libraries, but I intentionally decided to only use basic Python data structures.


In [0]:
!pip install -q xlsxwriter

In [0]:
import requests
import xlsxwriter
from lxml import etree

The following function takes an integer four-digit year as an argument and retrievs the daily Treasury yield curve rates for the specified year from the US Treasury website in XML format (the argument should be an integer between 1990 and 2019, as no data is available on the website for earlier years) The function then returns a 2-dimensional list with rates, where each column is a yield rate type and each row is a date (the first column contains dates in string format). \\
 \\

The ***xml_string*** contains the response from the website in string format. The ***tree*** is an ***Element*** object from the ***xml.etree*** module. The variables ***tbill_date***, ***tbill_1month***, ..., ***tbill_30year*** are all lists containing corresponding elements from the tree. In the concluding for loop, the text content of all these elements is retrieved and saved to a 2-dimensional list ***rates*** (each element is a list of length 13).

In [0]:
def get_rates(year):
    """ The function takes a four-digit numeric year (1990 - 2019) as an
    argument and retrievs the daily Treasury yield curve rates for the
    specified year from the US Treasury website in XML format. The function 
    then return a 2-dimensional list with rates where each column is a yield
    rate type and each row is a date."""

    if year not in range(1990,2020):
        print("Error: no data available for this year")
        return []
    
    url = "https://data.treasury.gov/feed.svc/DailyTreasuryYieldCurveRate" \
    + "Data?$filter=year(NEW_DATE)%20eq%20" + str(year)

    xml_string = requests.get(url).content
    
    tree = etree.fromstring(xml_string)
    
    tbill_date = tree.findall(".//{http://schemas.microsoft.com/ado/2007" \
                                   + "/08/dataservices}NEW_DATE")
    tbill_1month = tree.findall(".//{http://schemas.microsoft.com/ado/2007" \
                                     + "/08/dataservices}BC_1MONTH")
    tbill_2month = tree.findall(".//{http://schemas.microsoft.com/ado/2007" \
                                     + "/08/dataservices}BC_2MONTH")
    tbill_3month = tree.findall(".//{http://schemas.microsoft.com/ado/2007" \
                                     + "/08/dataservices}BC_3MONTH")
    tbill_6month = tree.findall(".//{http://schemas.microsoft.com/ado/2007" \
                                     + "/08/dataservices}BC_6MONTH")
    tbill_1year = tree.findall(".//{http://schemas.microsoft.com/ado/2007" \
                                    + "/08/dataservices}BC_1YEAR")
    tbill_2year = tree.findall(".//{http://schemas.microsoft.com/ado/2007" \
                                    + "/08/dataservices}BC_2YEAR")
    tbill_3year = tree.findall(".//{http://schemas.microsoft.com/ado/2007" \
                                    + "/08/dataservices}BC_3YEAR")
    tbill_5year = tree.findall(".//{http://schemas.microsoft.com/ado/2007" \
                                    + "/08/dataservices}BC_5YEAR")
    tbill_7year = tree.findall(".//{http://schemas.microsoft.com/ado/2007" \
                                    + "/08/dataservices}BC_7YEAR")
    tbill_10year = tree.findall(".//{http://schemas.microsoft.com/ado/2007" \
                                     + "/08/dataservices}BC_10YEAR")
    tbill_20year = tree.findall(".//{http://schemas.microsoft.com/ado/2007" \
                                     + "/08/dataservices}BC_20YEAR")
    tbill_30year = tree.findall(".//{http://schemas.microsoft.com/ado/2007" \
                                     + "/08/dataservices}BC_30YEAR")
    
    rates = []
    
    for n in range(len(tbill_date)):
        row = [tbill_date[n].text, tbill_1month[n].text, tbill_2month[n].text, \
              tbill_3month[n].text, tbill_6month[n].text, tbill_1year[n].text, \
              tbill_2year[n].text, tbill_3year[n].text, tbill_5year[n].text, \
              tbill_7year[n].text, tbill_10year[n].text, tbill_20year[n].text, \
              tbill_30year[n].text]
        
        rates.append(row)
        
    return rates

The ***start_year*** and ***end_year*** for the time period are arbitrary. As the list returned by the function has the same number of columns for all possible years, we can add the results iteratively to the ***rates*** list by simply using the ***+*** operator. This allows for a single 2-dimensional list containing rates for all years in the time period.

In [0]:
start_year = 2015
end_year = 2017

rates = []

for year in range(start_year, end_year + 1):
    rates += get_rates(year)

In what follows the list rates is written to an Excel file. I know this could be done easier with more advanced libraries like ***pandas***, but I was not able to figure out a more concise way to di it with just lists. The ***xlsxwriter*** library was chosen for no particular reason. \\
 \\
 
The ***{'strings_to_numbers':  True}*** parameter was added to the ***xlsxwriter.Workbook*** function to ensure that the rates currently in string format are written as numbers. The header with column names is added manually using ***write()*** function. Finally, all of the data is manually written to the file in a simple for loop.

In [0]:
file_name = "YieldCurveRates%s-%s.xlsx" % (start_year, end_year)
workbook = xlsxwriter.Workbook(file_name, {'strings_to_numbers':  True})
worksheet = workbook.add_worksheet()

worksheet.write('A1', 'Date')
worksheet.write('B1', '1 Mo')
worksheet.write('C1', '2 Mo')
worksheet.write('D1', '3 Mo')
worksheet.write('E1', '6 Mo')
worksheet.write('F1', '1 Yr')
worksheet.write('G1', '2 Yr')
worksheet.write('H1', '3 Yr')
worksheet.write('I1', '5 Yr')
worksheet.write('J1', '7 Yr')
worksheet.write('K1', '10 Yr')
worksheet.write('L1', '20 Yr')
worksheet.write('M1', '30 Yr')

row = 1
col = 0

for date, mo1, mo2, mo3, mo6, ye1, ye2, ye3, ye5, ye7, ye10, ye20, ye30 in \
rates:
    worksheet.write(row, col, date)
    worksheet.write(row, col + 1, mo1)
    worksheet.write(row, col + 2, mo2)
    worksheet.write(row, col + 3, mo3)
    worksheet.write(row, col + 4, mo6)
    worksheet.write(row, col + 5, ye1)
    worksheet.write(row, col + 6, ye2)
    worksheet.write(row, col + 7, ye3)
    worksheet.write(row, col + 8, ye5)
    worksheet.write(row, col + 9, ye7)
    worksheet.write(row, col + 10, ye10)
    worksheet.write(row, col + 11, ye20)
    worksheet.write(row, col + 12, ye30)
    
    row += 1
    
workbook.close()

I was able to save the file to my Google drive, but I do not feel comfortable including the code here as I have not figured out all the details about this process. The command for downloading the file to the local drive should be enough.

In [0]:
from google.colab import files
files.download(file_name)